In [1]:
import os
import json
from pathlib import Path
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from llama_index.readers.json import JSONReader
from llama_index.core.node_parser import JSONNodeParser
from llama_index.readers.file import FlatReader

In [2]:
embed_model = resolve_embed_model("local:BAAI/bge-m3")

In [3]:
# set a path to folder containing all the json files
DATA_PATH = "./data/"

# setting up reader, parser, and llm
reader = JSONReader()

# parser = JSONNodeParser()     # if we want to split the documents into nodes
llm = Ollama(model="llama3.2", request_timeout=180.0) 

In [4]:

# creating the documents out of the json files
documents = []
for filename in os.listdir(DATA_PATH):
    if filename.endswith(".json"):
        file_path = os.path.join(DATA_PATH, filename)
        documents.extend(FlatReader().load_data(Path(file_path)))     # if we want to load the data to then split it into nodes
        # documents.extend(reader.load_data(input_file=file_path))
parser = JSONNodeParser(include_metadata=True,
                        include_prev_next_rel=True)

# nodes = parser.get_nodes_from_documents(documents)            # if we want to split documents into nodes


In [5]:
len(documents)

6

### Document splitting

In [ ]:
#import nest_asyncio
#import nltk
#nltk.download('punkt_tab')
#nest_asyncio.apply()

[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [6]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor

from llama_index.core.node_parser import TokenTextSplitter

from llama_index.core.ingestion import IngestionPipeline


text_splitter = TokenTextSplitter(
    # separator=" ", 
    chunk_size=512, 
    chunk_overlap=128
)

#if you wanna create some custom extractor

# class CustomExtractor(BaseExtractor):
#     def extract(self, nodes):
#         metadata_list = [
#             {
#                 "custom": (
#                     node.metadata["document_title"]
#                     + "\n"
#                     + node.metadata["excerpt_keywords"]
#                 )
#             }
#             for node in nodes
#         ]
#         return metadata_list

transformations = [
    text_splitter,
    # TitleExtractor(nodes=3,llm=llm),
    # QuestionsAnsweredExtractor(questions=2,llm=llm),
    # SummaryExtractor(summaries=["prev", "self"],llm=llm),
    # KeywordExtractor(keywords=4,llm=llm),
    #EntityExtractor(prediction_threshold=0.5,llm=llm),
]


pipeline = IngestionPipeline(
    transformations=transformations
)

nodes = pipeline.run(
    documents=documents,
    in_place=True,
    show_progress=True,
)

/Users/lavo/Desktop/Projects/rag/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in EntityExtractor has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
len(nodes)

108

In [8]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding